# Project-3: Finsights Grey - RAG for Effective Information Retrieval


## Business Use Case

**Problem Statement:**

Finsights Grey Inc. is an innovative financial technology firm that specializes in providing advanced analytics and insights for investment management and financial planning. The company handles an extensive collection of 10-K reports from various industry players, which contain detailed information about financial performance, risk factors, market trends, and strategic initiatives. Despite the richness of these documents, Finsights Grey's financial analysts struggle with extracting actionable insights efficiently in a short span due to the manual and labor-intensive nature of the analysis. Going through the document to find the exact information needed at the moment takes too long. This bottleneck hampers the company's ability to deliver timely and accurate recommendations to its clients. To overcome these challenges, Finsights Grey Inc. aims to implement a Retrieval-Augmented Generation (RAG) model to automate the extraction, summarization, and analysis of information from the 10-K reports, thereby enhancing the accuracy and speed of their investment insights.

**Objective:**

As a Gen AI Data Scientist hired by Finsights Grey Inc., the objective is to develop an advanced RAG-based system to streamline the extraction and analysis of key information from 10-K reports. You are asked to deploy a Gradio app on HuggingFace spaces that can RAG 10-k reports and answer the questions of financial analysts swiftly.

The project will involve testing the RAG system on a current business problem. The Financial analysts are asked to research major cloud and AI platforms such as Amazon AWS, Google Cloud, Microsoft Azure, Meta AI, and IBM Watson to determine the most effective platform for this application. The primary goals include improving the efficiency of data extraction. Once the project is deployed, the system will be tested by a financial analyst with the following questions. Accurate text retrieval for these questions will imply the project's success.

**Questions:**

1. Has the company made any significant acquisitions in the AI space, and how are these acquisitions being integrated into the company's strategy?

2. How much capital has been allocated towards AI research and development?

3. What initiatives has the company implemented to address ethical concerns surrounding AI, such as fairness, accountability, and privacy?

4. How does the company plan to differentiate itself in the AI space relative to competitors?

Each Question must be asked for each of the five companies on the HuggingFace spaces.


**By successfully developing this project, we aim to:**

Improve the productivity of financial analysts by providing a competent tool.

Provide timely insights to improve client recommendations.

Strengthen FinTech Insights Inc.’s competitive edge by delivering more reliable and faster insights to clients.


**Connect to a T4 GPU Instance to create the Vector Database.**

### Setup

In [1]:
!pip -q install -U langchain langchain-community chromadb pymupdf openai gradio pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 131.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 162.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1

In [2]:
import os
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch


# Impementing RAG

### Prepare Data

Let's start by loading the dataset.

In [3]:
!unzip -o Dataset-10k.zip -d Dataset-10k

Archive:  Dataset-10k.zip
  inflating: Dataset-10k/IBM-10-k-2023.pdf  
  inflating: Dataset-10k/Meta-10-k-2023.pdf  
  inflating: Dataset-10k/aws-10-k-2023.pdf  
  inflating: Dataset-10k/google-10-k-2023.pdf  
  inflating: Dataset-10k/msft-10-k-2023.pdf  


## DB Creation

### Chunking

In [4]:
# Load PDFs (10-K reports) with page-level metadata
# NOTE: In Colab, upload/unzip the dataset first, or set DATA_DIR to your folder path.
import os
from pathlib import Path

DATA_DIR = Path("./Dataset-10k")  # <-- adjust if needed
assert DATA_DIR.exists(), f"DATA_DIR not found: {DATA_DIR.resolve()}"

pdf_files = sorted([p for p in DATA_DIR.glob("*.pdf")])
pdf_files


[PosixPath('Dataset-10k/IBM-10-k-2023.pdf'),
 PosixPath('Dataset-10k/Meta-10-k-2023.pdf'),
 PosixPath('Dataset-10k/aws-10-k-2023.pdf'),
 PosixPath('Dataset-10k/google-10-k-2023.pdf'),
 PosixPath('Dataset-10k/msft-10-k-2023.pdf')]

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Chunk sizes tuned for long-form filings (balance recall + token cost)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""],
)


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

def company_from_filename(path: Path) -> str:
    name = path.stem.lower()
    if "ibm" in name:
        return "IBM"
    if "meta" in name:
        return "Meta"
    if "aws" in name or "amazon" in name:
        return "Amazon (AWS)"
    if "google" in name or "alphabet" in name:
        return "Google"
    if "msft" in name or "microsoft" in name:
        return "Microsoft"
    # fallback
    return path.stem

all_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(str(pdf_path))
    docs = loader.load()  # one Document per page (metadata includes 'page')
    company = company_from_filename(pdf_path)
    for d in docs:
        # Add company + source to metadata so we can filter retrieval per company
        d.metadata["company"] = company
        d.metadata["source"] = pdf_path.name
    all_docs.extend(docs)

report_chunks = text_splitter.split_documents(all_docs)


In [7]:
len(report_chunks)


1978

In [8]:
report_chunks[0]


Document(metadata={'producer': 'Wdesk Fidelity Content Translations Version 009.008.020', 'creator': 'Workiva', 'creationdate': '2024-02-26T22:54:18+00:00', 'source': 'IBM-10-k-2023.pdf', 'file_path': 'Dataset-10k/IBM-10-k-2023.pdf', 'total_pages': 38, 'format': 'PDF 1.4', 'title': 'IBM-2023.12.31-10K-2024-02-26-17-54', 'author': 'anonymous', 'subject': '', 'keywords': '', 'moddate': '2024-03-01T15:23:24-05:00', 'trapped': '', 'modDate': "D:20240301152324-05'00'", 'creationDate': 'D:20240226225418Z', 'page': 0, 'company': 'IBM'}, page_content='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWASHINGTON, D.C. 20549\nFORM 10-K\nANNUAL REPORT\npursuant to Section 13 or 15 (d) of the\nSecurities Exchange Act of 1934\nFOR THE YEAR ENDED DECEMBER 31, 2023\n1-2360\n(Commission file number)\nINTERNATIONAL BUSINESS MACHINES CORPORATION\n(Exact name of registrant as specified in its charter)\nNew York \n(State of Incorporation) \n13-0871985\n(IRS Employer Identification Number)\nOne New Orchar

### Database Creation

In [9]:
COLLECTION_NAME = "finsights_10k_gte_large"
PERSIST_DIR = "./vector_db"


In [10]:
# Embeddings: thenlper/gte-large (as requested in rubric)
# In Colab: ensure GPU runtime if possible (T4). For CPU, it will be slower.
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings

device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)
device


/tmp/ipython-input-1458122476.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

'cuda'

In [11]:
from langchain_community.vectorstores import Chroma
import shutil, os

# Clean old DB if exists (optional)
if os.path.exists(PERSIST_DIR):
    print("Removing existing persisted DB...")
    shutil.rmtree(PERSIST_DIR)

vectordb = Chroma.from_documents(
    documents=report_chunks,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIR,
)

In [12]:
vectordb.persist()
print("Persisted to:", os.path.abspath(PERSIST_DIR))


Persisted to: /content/vector_db


/tmp/ipython-input-1152198291.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [13]:
# Mount Google Drive (Colab only)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print("Not running in Colab or Drive not available:", e)


Mounted at /content/drive


In [14]:
# Copy persisted DB to Google Drive so it can be reused later
# Adjust the destination path to your preferred folder.
import shutil, os
DRIVE_DB_DIR = "/content/drive/MyDrive/finsights_grey/vector_db"

if os.path.exists("/content/drive"):
    os.makedirs(os.path.dirname(DRIVE_DB_DIR), exist_ok=True)
    if os.path.exists(DRIVE_DB_DIR):
        shutil.rmtree(DRIVE_DB_DIR)
    shutil.copytree(PERSIST_DIR, DRIVE_DB_DIR)
    print("Copied persisted DB to:", DRIVE_DB_DIR)
else:
    print("Drive not mounted; skipping copy.")


Copied persisted DB to: /content/drive/MyDrive/finsights_grey/vector_db


# Retrieve DB from GDrive

###**Set up CPU Instance**

In [15]:
# If running on Colab CPU instance, install dependencies (skip if already installed)
!pip -q install -U "langchain>=0.2" langchain-community langchain-text-splitters chromadb pymupdf openai gradio pandas

In [16]:
from pathlib import Path
import torch
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 1) Mount Google Drive (only works in Google Colab)
try:
    from google.colab import drive
    drive.mount("/content/drive")
    DRIVE_MOUNTED = True
except Exception:
    DRIVE_MOUNTED = False

# 2) Choose where the persisted vector DB lives
COLLECTION_NAME = "finsights_10k_gte_large"
LOCAL_DB_DIR = "./vector_db"
DRIVE_DB_DIR = "/content/drive/MyDrive/finsights_grey/vector_db"

PERSIST_DIR = DRIVE_DB_DIR if (DRIVE_MOUNTED and os.path.exists(DRIVE_DB_DIR)) else LOCAL_DB_DIR
print("Using persisted DB folder:", PERSIST_DIR)

# 3) Initialise the embedding model (must match the one used when building the DB)
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)

# 4) Load the persisted Chroma vector database
vectordb = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIR,
    embedding_function=embeddings,
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using persisted DB folder: /content/drive/MyDrive/finsights_grey/vector_db


Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

BertModel LOAD REPORT from: thenlper/gte-large
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/tmp/ipython-input-3793622693.py:31: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(


In [17]:

import getpass
from openai import OpenAI

# Prompt securely (no key stored in the file)
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY (input hidden): ")

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
print("OPENAI_API_KEY is set for this session.")


Enter OPENAI_API_KEY (input hidden): ··········
OPENAI_API_KEY is set for this session.


In [18]:
# OpenAI credentials check (do NOT hardcode keys in the notebook)
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise RuntimeError(
        "OPENAI_API_KEY is not set. "
        "Set it in your environment (Colab Secrets, HF Secrets, or OS env vars)."
    )

client = OpenAI(api_key=api_key)

In [19]:
# Quick test: retrieve chunks for one company + one question

def retrieve_context(question: str, company: str, k: int = 4):
    """Find the top-k most similar chunks for a question, filtered by company."""
    retriever = vectordb.as_retriever(search_kwargs={"k": k, "filter": {"company": company}})
    return retriever.invoke(question)

def format_context(docs):
    """Turn retrieved docs into a single context string, including page numbers for citations."""
    lines = []
    for i, d in enumerate(docs, start=1):
        page = d.metadata.get("page", "?")
        src = d.metadata.get("source", "unknown")
        comp = d.metadata.get("company", "unknown")
        lines.append(f"[{i}] {comp} | {src} | page {page}\n{d.page_content}")
    return "\n\n".join(lines)

sample_question = "How much capital has been allocated towards AI research and development?"
sample_company = "Microsoft"

docs = retrieve_context(sample_question, sample_company, k=4)

print("Retrieved docs:", len(docs))

if docs:
    print(
        "First source/page:",
        docs[0].metadata.get("source"),
        docs[0].metadata.get("page")
    )
else:
    raise ValueError(
        f"No documents retrieved for company='{sample_company}'. "
        "Check metadata, filter value, or vector DB contents."
    )


Retrieved docs: 4
First source/page: msft-10-k-2023.pdf 1


In [20]:
docs = retrieve_context(sample_question, sample_company, k=4)

print("Retrieved docs:", len(docs))

if docs:
    print(
        "First source/page:",
        docs[0].metadata.get("source"),
        docs[0].metadata.get("page")
    )
else:
    raise ValueError(
        f"No documents retrieved for company='{sample_company}'. "
        "Check metadata, filter value, or vector DB contents."
    )


Retrieved docs: 4
First source/page: msft-10-k-2023.pdf 1


## RAG Q&A

### Prompt Design

In [21]:
# Prompt design: tell the LLM to answer ONLY from the retrieved context and cite pages.
qna_system_message = (
    "You are a careful financial analyst assistant. "
    "Answer the user's question using ONLY the provided CONTEXT from 10-K filings. "
    "If the answer is not present in the context, say you could not find it in the provided filings. "
    "Cite page numbers in the form (p. X) for every important claim."
)

qna_user_message_template = """QUESTION:
{question}

CONTEXT:
{context}

Write a helpful answer. Use citations like (p. 12). If context is insufficient, say so.
"""

In [22]:
# Build the messages we send to the OpenAI chat model
def build_messages(question: str, context: str):
    return [
        {"role": "system", "content": qna_system_message},
        {"role": "user", "content": qna_user_message_template.format(question=question, context=context)},
    ]

In [23]:
# Main RAG function: retrieve context -> build prompt -> call LLM -> return answer + sources
ANSWER_MODEL = os.getenv("ANSWER_MODEL", "gpt-4o-mini")

def answer_question(question: str, company: str, k: int = 4):
    # 1) Retrieve context from the vector DB (filtered by company)
    docs = retrieve_context(question, company, k=k)
    context = format_context(docs)

    # 2) Call the LLM with question + context
    resp = client.chat.completions.create(
        model=ANSWER_MODEL,
        messages=build_messages(question, context),
        temperature=0.2,
    )
    answer = resp.choices[0].message.content.strip()

    # 3) Collect sources (filename + page)
    sources = [(d.metadata.get("source"), d.metadata.get("page")) for d in docs]

    return {
        "company": company,
        "question": question,
        "answer": answer,
        "sources": sources,
        "n_context_docs": len(docs),
        "context": context,
    }

In [24]:
# Test on one sample question (Google)
test_out = answer_question(
    "Has the company made any significant acquisitions in the AI space, and how are these acquisitions being integrated into the company's strategy?",
    company="Google",
    k=4,
)
print(test_out["answer"][:1200])
print("\nSources:", test_out["sources"])

The provided filings do not specify any significant acquisitions in the AI space. However, they do indicate that the company is making substantial investments in AI across its operations, integrating AI capabilities into its products and services, and focusing on long-term strategies that include AI (p. 11, p. 30). The context mentions that acquisitions and strategic investments contribute to the breadth and depth of offerings and expand expertise, but it does not detail any specific acquisitions or how they are being integrated into the company's strategy (p. 30). Therefore, I could not find information on significant acquisitions in the AI space or their integration into the company's strategy.

Sources: [('google-10-k-2023.pdf', 30), ('google-10-k-2023.pdf', 11), ('google-10-k-2023.pdf', 12), ('google-10-k-2023.pdf', 6)]


# Evaluation

### Craft prompts for evaluation

In [25]:
# Rater / Judge model: usually a stronger model for evaluation
RATER_MODEL = os.getenv("RATER_MODEL", "gpt-4o-mini")  # adjust to what your endpoint supports


In [26]:
GROUNDEDNESS_SYSTEM = (
    "You are a strict evaluator. "
    "Check whether the ANSWER is fully supported by the CONTEXT. "
    "If the answer contains claims not found in the context, mark it as not grounded."
)

def groundedness_prompt(question: str, answer: str, context: str) -> list:
    user = f"""QUESTION:
{question}

CONTEXT:
{context}

ANSWER:
{answer}

Task:
Decide if the ANSWER is grounded in the CONTEXT.
Return ONLY one of: GROUNDED, PARTIALLY_GROUNDED, NOT_GROUNDED.
Then add one short sentence explaining why.
"""
    return [
        {"role": "system", "content": GROUNDEDNESS_SYSTEM},
        {"role": "user", "content": user},
    ]


In [27]:
RELEVANCE_SYSTEM = (
    "You are a strict evaluator. "
    "Check whether the ANSWER directly addresses the QUESTION, using the CONTEXT. "
    "Penalize vague or off-topic answers."
)

def relevance_prompt(question: str, answer: str, context: str) -> list:
    user = f"""QUESTION:
{question}

ANSWER:
{answer}

Task:
Decide if the ANSWER is relevant to the QUESTION.
Return ONLY one of: RELEVANT, PARTIALLY_RELEVANT, NOT_RELEVANT.
Then add one short sentence explaining why.
"""
    return [
        {"role": "system", "content": RELEVANCE_SYSTEM},
        {"role": "user", "content": user},
    ]


In [28]:
def call_rater(messages: list) -> str:
    resp = client.chat.completions.create(
        model=RATER_MODEL,
        messages=messages,
        temperature=0.0,
    )
    return resp.choices[0].message.content.strip()


### Test the Evaluation on One Sample

In [29]:
user_input = "How much capital has been allocated towards AI research and development?"
company_choice = "Microsoft"


In [30]:
# Retrieve context for this question (so we can evaluate groundedness and relevance)
docs = retrieve_context(user_input, company_choice, k=4)
context = format_context(docs)

In [31]:
answer_obj = answer_question(user_input, company=company_choice, k=4)
answer_text = answer_obj["answer"]


In [32]:
g_messages = groundedness_prompt(user_input, answer_text, context)
r_messages = relevance_prompt(user_input, answer_text, context)


In [33]:
print("\nGROUNDEDNESS RATING:")
print(call_rater(g_messages))



GROUNDEDNESS RATING:
GROUNDED.  
The answer accurately reflects the context by stating that specific monetary figures for AI capital allocation are not provided, while acknowledging the mention of significant investments in AI infrastructure.


In [34]:
print("\nRELEVANCE RATING:")
print(call_rater(r_messages))



RELEVANCE RATING:
PARTIALLY_RELEVANT.  
The answer addresses the question by indicating that specific monetary figures are not provided, but it lacks a direct response to the inquiry about the amount of capital allocated.


In [35]:
# (Optional) Inspect sources used
answer_obj["sources"][:5]


[('msft-10-k-2023.pdf', 1),
 ('msft-10-k-2023.pdf', 23),
 ('msft-10-k-2023.pdf', 38),
 ('msft-10-k-2023.pdf', 5)]

### Evaluation on multiple-queries

In [36]:
# Use 3 queries to produce 15 rows (3 queries x 5 companies = 15), as required by rubric.
queries = [
    "Has the company made any significant acquisitions in the AI space, and how are these acquisitions being integrated into the company's strategy?",
    "How much capital has been allocated towards AI research and development?",
    "What initiatives has the company implemented to address ethical concerns surrounding AI, such as fairness, accountability, and privacy?",
]

companies = ["Amazon (AWS)", "Google", "Microsoft", "Meta", "IBM"]

In [37]:
# Faster evaluation loop: reduces judge calls from 2 → 1 per (query, company)
# We ask the judge for BOTH groundedness and relevance in a single response, then parse it.

import re
import pandas as pd

def combined_judge_messages(question: str, answer: str, context: str):
    return [
        {
            "role": "system",
            "content": (
                "You are a strict evaluator for a RAG system. "
                "Score the answer ONLY using the provided context."
            ),
        },
        {
            "role": "user",
            "content": f"""
QUESTION:
{question}

ANSWER:
{answer}

CONTEXT:
{context}

Return EXACTLY two lines in this format:
GROUNDEDNESS: <0-5>
RELEVANCE: <0-5>

(0 = very poor, 5 = excellent)
""".strip(),
        },
    ]

def parse_two_scores(judge_text: str):
    # Defaults if parsing fails
    groundedness = None
    relevance = None

    g_match = re.search(r"GROUNDEDNESS:\s*([0-5])", judge_text, re.IGNORECASE)
    r_match = re.search(r"RELEVANCE:\s*([0-5])", judge_text, re.IGNORECASE)

    if g_match:
        groundedness = int(g_match.group(1))
    if r_match:
        relevance = int(r_match.group(1))

    return groundedness, relevance

rows = []

for q in queries:
    for comp in companies:
        # 1) Get the answer (your existing function)
        ans = answer_question(q, company=comp, k=4)

        # 2) Retrieve context ONCE for judging (you can drop k to 2 to cut tokens/cost further)
        docs = retrieve_context(q, company=comp, k=4)
        ctx = format_context(docs)

        # 3) ONE judge call instead of two
        judge_text = call_rater(combined_judge_messages(q, ans["answer"], ctx))
        g_score, r_score = parse_two_scores(judge_text)

        rows.append({
            "company": comp,
            "query": q,
            "answer": ans["answer"],
            "groundedness": g_score,
            "relevance": r_score,
            "judge_raw": judge_text,            # keep raw output for debugging/auditing
            "n_context_docs": ans["n_context_docs"],
            "sources": ans["sources"],
        })

eval_df = pd.DataFrame(rows)
eval_df.shape, eval_df.head(2)



((15, 8),
         company                                              query  \
 0  Amazon (AWS)  Has the company made any significant acquisiti...   
 1        Google  Has the company made any significant acquisiti...   
 
                                               answer  groundedness  relevance  \
 0  I could not find any information regarding sig...             3          2   
 1  The provided context does not specify any sign...             3          4   
 
                          judge_raw  n_context_docs  \
 0  GROUNDEDNESS: 3  \nRELEVANCE: 2               4   
 1  GROUNDEDNESS: 3  \nRELEVANCE: 4               4   
 
                                              sources  
 0  [(aws-10-k-2023.pdf, 19), (aws-10-k-2023.pdf, ...  
 1  [(google-10-k-2023.pdf, 30), (google-10-k-2023...  )

You might experience some hallucination in LLM's response. Try to change your prompt to mitigate this. Selecting a good model will also help mitigating hallucination, increase groundedness and relevance.

# Gradio Interface

In [45]:
%%writefile app.py
import os
from pathlib import Path
import gradio as gr
import torch
from openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

COLLECTION_NAME = "finsights_10k_gte_large"
PERSIST_DIR = "./vector_db"

ANSWER_MODEL = os.getenv("ANSWER_MODEL", "gpt-4o-mini")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or os.getenv("ANYSCALE_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL") or os.getenv("ANYSCALE_BASE_URL")

if not OPENAI_API_KEY:
    raise RuntimeError(
        "Missing API key. Set OPENAI_API_KEY (or ANYSCALE_API_KEY) as an environment secret."
    )

client = OpenAI(
    api_key=OPENAI_API_KEY,
    base_url=OPENAI_BASE_URL,   # None is fine if not using a custom base_url
)





device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)

vectordb = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIR,
    embedding_function=embeddings,
)

SYSTEM_PROMPT = (
    "You are a careful financial analyst assistant. "
    "Answer using ONLY the provided context from 10-K filings. "
    "If the answer is not in the context, say you could not find it. "
    "Always cite page numbers in the form (p. X)."
)

def retrieve_context(query: str, company: str, k: int = 4):
    """
    Retrieve top-k relevant chunks for a query, filtered by company.
    Compatible with both old and new LangChain retriever APIs.
    """

    # Use existing vectordb (do NOT recreate it)
    retriever = vectordb.as_retriever(
        search_kwargs={"k": int(k), "filter": {"company": company}}
    )

    # New LangChain retriever API (>=0.1.x / 0.2.x)
    if hasattr(retriever, "invoke"):
        return retriever.invoke(query)

    # Older LangChain fallback
    if hasattr(retriever, "get_relevant_documents"):
        return retriever.get_relevant_documents(query)

    # Defensive fail (should never happen)
    raise RuntimeError(
        "Unsupported LangChain retriever version: "
        "no invoke() or get_relevant_documents() found."
    )
def format_context(docs):
    parts = []
    for i, d in enumerate(docs, start=1):
        page = d.metadata.get("page", None)
        src = d.metadata.get("source", "unknown")
        company = d.metadata.get("company", "unknown")
        header = f"[{i}] Company: {company} | Source: {src} | Page: {page}"
        parts.append(header + "\n" + d.page_content)
    return "\n\n".join(parts)

def build_messages(question: str, context: str):
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"QUESTION:\n{question}\n\nCONTEXT:\n{context}\n\nANSWER:"},
    ]

def predict(company: str, question: str, top_k: int):
    if not question or not question.strip():
        return "Please enter a question."
    docs = retrieve_context(question, company=company, k=int(top_k))
    context = format_context(docs)
    resp = client.chat.completions.create(
        model=ANSWER_MODEL,
        messages=build_messages(question, context),
        temperature=0.2,
    )
    answer = resp.choices[0].message.content
    # Append sources at end
    sources = "\n".join([f"- {d.metadata.get('source')} (p. {d.metadata.get('page')})" for d in docs])
    return f"{answer}\n\nSources:\n{sources}"

companies = ["Amazon (AWS)", "Google", "Microsoft", "Meta", "IBM"]

with gr.Blocks(title="Finsights Grey — 10-K RAG Q&A") as demo:
    gr.Markdown(
        """
        # 📄 Finsights Grey — 10-K RAG Q&A
        Select a company, ask a question, and get an answer grounded in the 10-K report.
        """
    )

    with gr.Row():
        company_in = gr.Radio(choices=companies, value="Google", label="Select Company")
        top_k = gr.Slider(1, 8, value=4, step=1, label="Top-K retrieved chunks")

    question_in = gr.Textbox(lines=3, label="Your question")
    answer_out = gr.Textbox(lines=14, label="Answer")

    gr.Button("Ask").click(fn=predict, inputs=[company_in, question_in, top_k], outputs=[answer_out])

if __name__ == "__main__":
    demo.launch()



Writing app.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [39]:
demo.launch(share=True, debug=False)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://48916f2134a5f77901.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://48916f2134a5f77901.gradio.live


In [41]:
import json
import sys
import subprocess
from pathlib import Path

# Path to your notebook
NOTEBOOK_PATH = "/content/drive/MyDrive/Colab Notebooks/Project3_FinsightsGrey_RAG_cleaned (1).ipynb"


# Load notebook
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as f:
    nb = json.load(f)

# Collect imported top-level packages
packages = set()

for cell in nb.get("cells", []):
    if cell.get("cell_type") == "code":
        for line in cell.get("source", []):
            line = line.strip()

            if line.startswith("import "):
                pkg = line.split()[1].split(".")[0]
                packages.add(pkg)

            elif line.startswith("from "):
                pkg = line.split()[1].split(".")[0]
                packages.add(pkg)

# Remove standard-library modules
stdlib = {
    "os", "sys", "pathlib", "json", "shutil",
    "subprocess", "typing", "math", "time"
}
packages = packages - stdlib

# Helper to get installed version
def get_version(pkg):
    try:
        return subprocess.check_output(
            [sys.executable, "-m", "pip", "show", pkg],
            stderr=subprocess.DEVNULL,
            text=True
        ).split("Version: ")[1].splitlines()[0]
    except Exception:
        return "NOT INSTALLED"

# Print requirements.txt style output
print("\n# === requirements.txt (auto-generated) ===\n")
for pkg in sorted(packages):
    version = get_version(pkg)
    print(f"{pkg}=={version}")


# === requirements.txt (auto-generated) ===

google==3.0.0
gradio==6.5.1
langchain_community==0.4.1
langchain_text_splitters==1.1.0
openai==2.20.0
pandas==3.0.0
re==NOT INSTALLED
shutil,==NOT INSTALLED
torch==2.9.0+cu128


In [42]:
%%writefile requirements.txt

gradio

langchain-community
langchain-text-splitters
chromadb==1.4.1
sentence-transformers==2.3.1

openai==1.57.4
pandas==2.2.3
numpy==1.26.4
torch==2.2.2
httpx==0.27.2



Writing requirements.txt


In [43]:
import chromadb
print(chromadb.__version__)


1.5.0


In [ ]:
# : Upload /content/vector_db to your HF Space (no git, no LFS)

import os
import getpass
from huggingface_hub import HfApi

SPACE_REPO = "dariuscyrus7819/edwin_10k_reports_qna"   # your Space repo
LOCAL_FOLDER = "/content/vector_db"                   # your Colab folder
TARGET_PATH = "vector_db"                             # folder name in the Space repo

# 1) Install dependency (quiet)
!pip -q install -U huggingface_hub

# 2) Get token securely (hidden input). Use a WRITE token from HF settings.
if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass.getpass("Paste your HF WRITE token (hidden): ")

# 3) Upload the whole folder to the Space repo
api = HfApi()
api.upload_folder(
    repo_id=SPACE_REPO,
    repo_type="space",
    folder_path=LOCAL_FOLDER,
    path_in_repo=TARGET_PATH,
    token=os.environ["HF_TOKEN"],
)

print(" Uploaded /content/vector_db to the Space as /vector_db")


## Deployment to Hugging Face Spaces (Gradio)

1. Create a **public** Gradio Space at your profile (example naming: `yourname_reports_qna`).
2. Upload these items to the Space repo:
   - `app.py`
   - `requirements.txt`
   - `vector_db/` (the persisted Chroma DB folder)
3. Add secrets (Settings → Secrets):
   - `OPENAI_API_KEY` **or** `ANYSCALE_API_KEY`
   - optionally `OPENAI_BASE_URL` / `ANYSCALE_BASE_URL`
   - optionally `ANSWER_MODEL` (default in code is `gpt-4o-mini`)
4. After build completes, open the Space and test:
   - Select different companies with the radio button
   - Ask the evaluation questions; outputs should differ when context exists.


### Paste your gradio app link and logs link

*   app link here

*   logs_dataset link here

Note: Make sure your Hugging Face space repository and the logs_dataset are set to public. If it's private, the evaluator won't be able to access the app you've built, which could result in losing marks.

# Convert ipynb to HTML

Instructions:
1. Go to File
2. Download these current working Notebook in to ipynb format
3. Now, run the below code, select the notebook from local where you downloaded the file
4. Wait for few sec, your notebook will automatically converted in to html format and save in your local pc


In [44]:
from pathlib import Path
import nbformat
import subprocess, sys

src = Path("/content/drive/MyDrive/Colab Notebooks/Project3_FinsightsGrey_RAG_cleaned (1).ipynb")
dst = src.with_name(src.stem + "_nowidgets.ipynb")

nb = nbformat.read(src, as_version=4)

for cell in nb.cells:
    if cell.get("cell_type") == "code":
        # remove outputs that can trigger widget-state conversion failures
        cell["outputs"] = []
        cell["execution_count"] = None

# also remove widget metadata if present
nb.metadata.pop("widgets", None)

nbformat.write(nb, dst)

subprocess.run([sys.executable, "-m", "pip", "install", "-qU", "nbconvert", "jupyter"], check=True)
subprocess.run([sys.executable, "-m", "jupyter", "nbconvert", str(dst), "--to", "html"], check=True)

print(" Clean HTML created from:", dst)
print("HTML:", dst.with_suffix(".html"))


 Clean HTML created from: /content/drive/MyDrive/Colab Notebooks/Project3_FinsightsGrey_RAG_cleaned (1)_nowidgets.ipynb
HTML: /content/drive/MyDrive/Colab Notebooks/Project3_FinsightsGrey_RAG_cleaned (1)_nowidgets.html


Gradio link address on colab
https://b568ae1db08946c50f.gradio.live/

Gradio link address on Hugging Face Space:
https://dariuscyrus7819-edwin-10k-reports-qna.hf.space/

## Power Ahead!